<a href="https://www.quantrocket.com"><img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png"></a><br>
<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

***
[FX Business Day Strategy](Introduction.ipynb) › Part 8: Scheduling
***

# Scheduling

To automate trading of the `fx-bizday` strategy, the remaining step is to schedule data to be collected and Moonshot to run at the appropriate times. Running cron jobs is the task of the `countdown` service. The example commands to run are provided in [quantrocket.countdown.crontab.sh](quantrocket.countdown.crontab.sh).

## Crontab overview

The cron job for live trading runs at 3 AM, 11 AM, and 4 PM and consists of the following commands:

* Update the FX database with the latest data
* Wait for data collection to finish
* Run Moonshot to generate the latest orders and send them to the blotter

We use the priority queue to collect the data in case any longer-running data collections are in progress. Also, we sleep a few seconds first to ensure that the 3AM/11AM/4PM data collection includes a 3AM/11AM/4PM bar, as otherwise Moonshot will complain that our data is stale. (See the usage guide to learn more about trade time validation for intraday Moonshot strategies.)

```shell
# Update historical database, wait for it to finish, then trade fx-bizday each weekday at 3 AM, 11 AM, and 4 PM. This command "paper trades" 
# fx-bizday by logging orders to flightlog; to live or paper trade with broker, send orders to blotter instead:
#     ... quantrocket moonshot trade 'fx-bizday' | quantrocket blotter order -f -
0 3,11,16 * * mon-fri sleep 3 && quantrocket history collect 'fiber-1h' --priority && quantrocket history wait 'fiber-1h' && quantrocket moonshot trade 'fx-bizday' | quantrocket flightlog log -n 'quantrocket.moonshot'
```

> In this example we are using an up-to-date history database as our real-time data source for live trading. This is a simple approach and often effective when only trading a small number of securities. For other use cases, real-time data aggregated to the bar size of the strategy may be preferable. Learn more in the [usage guide](https://www.quantrocket.com/docs/#realtime).

## Install the crontab

> This section assumes that you're not already using your `countdown` service for any scheduled tasks and that you haven't yet set its timezone. If you're already using `countdown`, you can edit your existing crontab, or add a new countdown service for New York tasks. See the usage guide for help.  

All the commands on the provided crontab are intended to be run in New York time. By default, the countdown timezone is UTC:

In [1]:
from quantrocket.countdown import get_timezone, set_timezone
get_timezone()

{'timezone': 'UTC'}

So first, set the countdown timezone to New York time:

In [2]:
set_timezone("America/New_York")

{'status': 'successfully set timezone to America/New_York'}

Install the crontab by moving it to the `/codeload` directory. (First open a flightlog terminal so you can see if it loads successfully.)

In [3]:
# move file over unless it already exists
![ -e /codeload/quantrocket.countdown.crontab* ] && echo 'oops, the file already exists!' || mv quantrocket.countdown.crontab.sh /codeload/

You should see a success message in flightlog:

```
quantrocket.countdown: INFO Successfully loaded quantrocket.countdown.crontab.sh
```

***

[Back to Introduction](Introduction.ipynb)